In [ ]:
import pandas as pd
from os import walk

In [ ]:
datasets_names = []
path_df = '../datasets/original_files/'
for (dirpath, dirnames, filenames) in walk(path_df):
    datasets_names.extend(filenames)
    break
datasets_names

In [ ]:
dataFrame = False
first = True
for name in datasets_names:
    if(first):
        first = False
        dataFrame = pd.read_csv(path_df+name, compression='gzip')
        dataFrame.set_index('id', drop=False, inplace=True)
        print(name,'adding', len(dataFrame), 'rows')
    else:
        _tmpDf = pd.read_csv(path_df+name, compression='gzip')
        _tmpDf.set_index('id', drop=False, inplace=True)
        print(name,'adding', len(_tmpDf), 'rows')
        dataFrame = pd.concat([dataFrame, _tmpDf])
        del _tmpDf
print('total rows:', len(dataFrame))
dataFrame.head()

# Categorizando Columnas

In [ ]:
text_cols = ["name","summary","space","description","neighborhood_overview","notes","transit","access",
             "interaction","house_rules","host_about"];
dummyable_cols = ["instant_bookable","is_business_travel_ready","cancellation_policy",
                  "require_guest_phone_verification","requires_license",
                  "require_guest_profile_picture","host_location","host_response_time",
                  "host_is_superhost","host_neighbourhood","host_has_profile_pic","host_identity_verified",
                  "has_availability","neighbourhood_cleansed","city","state","market",
                  "country_code","property_type","room_type","bed_type"];
dropeable_cols = ["id","listing_url","scrape_id","last_scraped","thumbnail_url","medium_url",
                  "picture_url","xl_picture_url","host_url","host_name","host_id","host_thumbnail_url",
                 "host_picture_url","host_total_listings_count","calendar_updated","calendar_last_scraped",
                  "neighbourhood","neighbourhood_group_cleansed","zipcode","smart_location","country",
                 "experiences_offered"];
continuos_cols = ["reviews_per_month","host_response_rate","host_acceptance_rate",
                  "review_scores_communication","review_scores_location","review_scores_value",
                  "number_of_reviews_ltm","review_scores_rating","review_scores_cleanliness",
                  "review_scores_checkin","availability_30","availability_60","availability_90","availability_365",
                 "review_scores_accuracy","minimum_nights","maximum_nights","minimum_minimum_nights",
                  "maximum_minimum_nights","minimum_maximum_nights","maximum_maximum_nights",
                  "minimum_nights_avg_ntm","maximum_nights_avg_ntm",
                  "calculated_host_listings_count","calculated_host_listings_count_entire_homes",
                  "calculated_host_listings_count_private_rooms","calculated_host_listings_count_shared_rooms",
                  "host_listings_count","number_of_reviews",
                  "accommodates","bathrooms","bedrooms","beds","guests_included",
                  ]
date_cols = ["host_since","first_review","last_review"]
explotable_cols = ["street","host_verifications","jurisdiction_names","amenities"]
geo_cols = ["latitude","longitude","is_location_exact"]

target_col = ["price","weekly_price","monthly_price","security_deposit","cleaning_fee","extra_people"]
#license ?

In [ ]:
#dataFrame.drop(text_cols,axis=1,inplace=True);
#dataFrame.drop(dummyable_cols,axis=1,inplace=True);
#dataFrame.drop(dropeable_cols,axis=1,inplace=True);
#dataFrame.drop(continuos_cols,axis=1,inplace=True);
#dataFrame.drop(date_cols,axis=1,inplace=True);
#dataFrame.drop(explotable_cols,axis=1,inplace=True);
#dataFrame.drop(geo_cols,axis=1,inplace=True);

#dataFrame.drop(target_col,axis=1,inplace=True);

# Ejemplos de las columnas categorizadas

In [ ]:
#Columnas detectadas como texto
dataFrame.sample(5).loc[:,text_cols].T

In [ ]:
#Columnas detectadas como candidatas para hacer Dummies
dataFrame.sample(5).loc[:,dummyable_cols].T

In [ ]:
#Columnas a ser dropeadas directamente
dataFrame.sample(5).loc[:,dropeable_cols].T

In [ ]:
#Columnas detectadas como valores continuos, para reescalar
dataFrame.sample(5).loc[:,continuos_cols].T

In [ ]:
#Columnas detectadas como fecha 
dataFrame.sample(5).loc[:,date_cols].T

In [ ]:
#Columnas que habria que trabajar para separarles los datos
dataFrame.sample(5).loc[:,explotable_cols].T

In [ ]:
#Columnas detectadas como informacion de geolocalizacion
dataFrame.sample(5).loc[:,geo_cols].T

In [ ]:
#Columnas detectadas como que pueden filtrar informacion del target
dataFrame.sample(5).loc[:,target_col].T

# Test de modelado solo con Dummies

In [ ]:
# drop outliers
dataFrame["price"]=dataFrame["price"].str.replace('$', '').str.replace(',', '').astype(float)

Q1 = dataFrame["price"].quantile(0.02)
Q3 = dataFrame["price"].quantile(0.98)

dataFrame.drop(dataFrame[dataFrame["price"] > Q3].index,axis=0,inplace=True);
dataFrame.drop(dataFrame[dataFrame["price"] < Q1].index,axis=0,inplace=True);

dataFrame["price"].describe()

In [ ]:
import numpy as np

# generamos las dummies y limpiamos el vector target
X = pd.get_dummies(dataFrame[dummyable_cols].astype(str), drop_first=True).reset_index(drop=True)
y = np.log(dataFrame["price"].reset_index(drop=True));
X.shape

In [ ]:
y.describe()

In [ ]:
# separamos entrenamiento y test
from sklearn.model_selection import train_test_split
train_data, val_data, train_targets, val_targets = train_test_split(X, y, test_size=0.2, random_state=42)

train_data.shape

In [ ]:
train_targets

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers

model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(train_data.shape[1],),kernel_regularizer=regularizers.l1(0.001)))
model.add(Dense(50, activation='relu',kernel_regularizer=regularizers.l1(0.001)))
model.add(Dense(50, activation='relu',kernel_regularizer=regularizers.l1(0.001)))
model.add(Dense(1,kernel_regularizer=regularizers.l1(0.001)))
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mae'])
model.fit(train_data, train_targets,epochs=5, batch_size=500, verbose=1)

In [ ]:
model.evaluate(val_data, val_targets, verbose=0)

# Inspeccionamos los valores continuos

In [ ]:
#revisamos una pequeña muestra de los datos
dataFrame.sample(5).loc[:,continuos_cols].T

* 1ro lo obvio -> son todos valores continuos.
* 2do -> Se puede apreciar presencia de valores null.
* 3ro -> Se vé que hay diferentes escalas de datos. Será conveniente reescalar y normalizar.
* 4to -> Quizas sea favorable puede aplicar transformaciones para favorecer la predictibilidad


In [ ]:
dataFrame.loc[:,continuos_cols].info()

* Acá se detectan 2 columnas que vienen con tipo de dato Object

In [ ]:
# Analisis clasico
dataFrame.loc[:,continuos_cols].describe().T

* se detectan varios campos con valores null, vamos a usar esto para intentar rellenarlo cuando sea posible

# Graficamos para ver interelacion

In [ ]:
import seaborn as sns; sns.set(style="ticks", color_codes=True)
sns.pairplot(dataFrame.sample(1000).loc[:,continuos_cols])

# Creamos los pasos del pipeline

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

#Esta clase simplemente filtra las columnas que se le indica en el constructor
class FeatureSelector( BaseEstimator, TransformerMixin ):
    #Class Constructor 
    def __init__( self, feature_names ):
        self._feature_names = feature_names 
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        return self 
    
    #Method that describes what we need this transformer to do
    def transform( self, X, y = None ):
        return X[ self._feature_names ] 

In [ ]:
import numpy as np

class NumericalImputer(BaseEstimator, TransformerMixin):
    def __init__( self, default_strategy = "median"):
        self._default_strategy = default_strategy
        self._default_values = {}
        
    def fit( self, X, y = None ):
        X.host_response_rate = X.host_response_rate.str.replace('%', '').astype(float)
        X.host_acceptance_rate = X.host_acceptance_rate.str.replace('%', '').astype(float)
        
        #Si hay valores infinitos los convertimos en NaN
        X = X.replace( [ np.inf, -np.inf ], np.nan )
        
        for col in X.columns:
            if col=='number_of_reviews_ltm':
                default_value=0;
            elif col=='number_of_reviews':
                default_value=0;
            elif col=='host_listings_count':
                default_value=1;
            elif self._default_strategy=='median':
                default_value=np.median(X[col].dropna())
            elif self._default_strategy=='mode':
                default_value=np.mode(X[col].dropna())
            elif self._default_strategy=='mean':
                default_value=np.mean(X[col].dropna())
            else:
                default_value=np.median(X[col].dropna())
            self._default_values[col]=default_value

        return self 
    
    def transform(self, X, y = None):
        X.host_response_rate = X.host_response_rate.astype(str).str.replace('%', '').astype(float)
        X.host_acceptance_rate = X.host_acceptance_rate.astype(str).str.replace('%', '').astype(float)
        
        for col in X.columns:
            #Si hay valores infinitos los convertimos en NaN
            X[col] = X[col].replace( [ np.inf, -np.inf ], np.nan)
            X[col].fillna(self._default_values[col],inplace=True)
        return X

class NumericalTransformer(BaseEstimator, TransformerMixin):
    def __init__( self, log_transform = True, bath_per_bed = True, bath_per_bedroom = True ):
        self._bath_per_bed = bath_per_bed
        self._bath_per_bedroom = bath_per_bedroom
        self._log_transform = log_transform
        
    def fit( self, X, y = None ):
        return self 
    
    def transform(self, X, y = None):
        if self._bath_per_bedroom:
            X.loc['bath_per_bedroom'] = X['bathrooms'] / X['bedrooms']
        if self._bath_per_bed:
            X.loc['bath_per_bed'] = X['bathrooms'] / X['beds']
        if self._log_transform:
            for col in X.columns:
                X[col] = X[col];
                #X[col] = np.log(X[col])
        
        #pd.cut(df.Age,bins=[0,2,17,65,99],labels=['Toddler/Baby','Child','Adult','Elderly'])
        
        #Retornamos un array de Numpy
        return X

# Generamos los pipelines

In [ ]:
from sklearn.pipeline import FeatureUnion, Pipeline 
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#Definimos los pasos para el pipeline de Dummies
#categorical_pipeline = Pipeline( steps = [ ( 'cat_selector', FeatureSelector(categorical_features) ),
#                                  ( 'cat_transformer', CategoricalTransformer() ), 
#                                  ( 'one_hot_encoder', OneHotEncoder( sparse = False ) ) ] )
    
#Definimos los pasos para el pipeline Numerico
numerical_pipeline = Pipeline( steps = [ ( 'num_selector', FeatureSelector(continuos_cols) ),
                                  ( 'num_imputer', NumericalImputer(default_strategy = 'median') ),
                                  ( 'num_transformer', NumericalTransformer() ),
                                  ( 'std_scaler', StandardScaler() ) 
                                       ] )

#Combinamos los diferentes pipelines en uno gran pipeline horizontal usando FeatureUnion
full_pipeline = FeatureUnion( transformer_list = [ 
#                                                  ( 'categorical_pipeline', categorical_pipeline ), 
                                                  ( 'numerical_pipeline', numerical_pipeline ) ] )

In [ ]:
full_pipeline.fit(dataFrame);

In [ ]:
result=pd.DataFrame( full_pipeline.transform(dataFrame))

In [ ]:
result